# A* algoritm for set covering problem

Here is the solution of the set covering problem.
The A* is searching algorithm which aims to find the shortest path between an initial and a final state.
This algorithm is based on the estimation of the total cost from the initial to the final state for each state reached.
So for each state are computed a cost function, which represent the cost to reach that state by the initial state, and the heuristic function which estimates in an optimistical way the cost to reach the final state.

The cost function is the amount of sets select up until the current state, because of the unit cost of each set.
Here I proposed two heuristics:
- heuristic1(state) estimate the cost by computing the number of uncovered elements of the current set and the ordered list of the number of covered elements of the not taken sets which are in the same position of uncovered elements in the current set and returning as a result the number of sets which sum of the dimensions of covered elements is higher than the uncovered size of the current set.
- heuristic2(state) estimate the cost by computing the list of the not taken sets ordered by the number of covered elements which are in the same position of uncovered elements in the current set and trying to fill the current set with the first set of the ordered list.
  
  So the current set is updated with the covered elements of the taken set and then the list of not taken sets is recomputed and so on, until the actual set is entirely covered and so the number of the added sets is the heuristic result.

In [186]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
import numpy as np
from tqdm.auto import tqdm


In [187]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

In [188]:
PROBLEM_SIZE = 10
NUM_SETS = 100
State = namedtuple('State', ['taken', 'not_taken'])
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [189]:
def heuristic1(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    empty_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < empty_size:
        taken += 1
    return taken

def heuristic2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    candidates = sorted([tuple([sum(np.logical_and(s, np.logical_not(already_covered))), s]) for s in SETS], key=lambda a:a[0], reverse=True)
    
    h = 0
    while not np.all(already_covered):
        already_covered = np.logical_or(candidates[h][1], already_covered)
        candidates = sorted([tuple([sum(np.logical_and(s, np.logical_not(already_covered))), s]) for s in SETS], key=lambda a:a[0], reverse=True)
        h += 1

    return h

def cost(state):
    return len(state.taken)

def f1(state):
    return cost(state) + heuristic1(state)

def f2(state):
    return cost(state) + heuristic2(state)


In [190]:
def solve(f):
    frontier = PriorityQueue()
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((f(state), state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken | {action},
                current_state.not_taken - {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
    print(f"Solved in {counter:} steps ({len(current_state.taken)} sets taken)")
    return counter, len(current_state.taken)



In [191]:
ITER_NUM = 500
counter1_sum = 0
counter2_sum = 0
for i in range(ITER_NUM):
    global SETS
    SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
    counter1, n_sets1 = solve(f1)
    counter2, n_sets2 = solve(f2)
    counter1_sum += counter1
    counter2_sum += counter2
    if n_sets1 != n_sets2:
        print(f"Error! {n_sets1:}, {n_sets2:}")

print(f"Average steps number with heuristic1: {counter1_sum/ITER_NUM}")
print(f"Average steps number with heuristic2: {counter2_sum/ITER_NUM}")


Solved in 26 steps (3 sets taken)
Solved in 11 steps (3 sets taken)
Solved in 24 steps (3 sets taken)
Solved in 14 steps (3 sets taken)
Solved in 18 steps (3 sets taken)
Solved in 12 steps (3 sets taken)
Solved in 24 steps (3 sets taken)
Solved in 9 steps (3 sets taken)
Solved in 13 steps (3 sets taken)
Solved in 15 steps (3 sets taken)
Solved in 34 steps (3 sets taken)
Solved in 9 steps (3 sets taken)
Solved in 15 steps (3 sets taken)
Solved in 12 steps (3 sets taken)
Solved in 43 steps (3 sets taken)
Solved in 14 steps (3 sets taken)
Solved in 13 steps (3 sets taken)
Solved in 9 steps (3 sets taken)
Solved in 31 steps (3 sets taken)
Solved in 8 steps (3 sets taken)
Solved in 34 steps (3 sets taken)
Solved in 14 steps (3 sets taken)
Solved in 23 steps (3 sets taken)
Solved in 21 steps (3 sets taken)
Solved in 18 steps (3 sets taken)
Solved in 13 steps (3 sets taken)
Solved in 19 steps (3 sets taken)
Solved in 10 steps (3 sets taken)
Solved in 19 steps (3 sets taken)
Solved in 12 steps

## Results
Here is a test running 500 times the two heuristics with the parameters below.

- PROBLEM_SIZE = 20
- NUM_SETS = 100

Output:

```
Average steps number with heuristic1: 24.542
Average steps number with heuristic2: 13.042
```